In [ ]:
!pip install fuzzywuzzy
!pip install python-Levenshtein
!pip install xlsxwriter

     |████████████████████████████████| 50 kB 2.8 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149858 sha256=5972da6908ac50a2a2dd0de39c8673e9cfea18a422dc2d5f19f7956bed12dc8c
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein
     |████████████████████████████████| 149 kB 4.4 MB/s 


In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pandas as pd
from google.colab import files

In [ ]:
site_data_report = files.upload()
site_data_file = next(iter(site_data_report))
df = pd.read_excel(site_data_file)

Saving MyROR Site Data Report 2022-02-22.xlsx to MyROR Site Data Report 2022-02-22.xlsx


In [ ]:
state = input('Enter the abbreviations of the state you want to look up: ')
# Filter by state
df = df[df['Physical Location: State'] == state]

# Select high level active only sites
high_level_active = ['Active', 'Active - Non-Reporting', 'Slated for deactivation', 'COVID-19 Freeze'] 
df = df[df['Status'].isin(high_level_active)] 


Enter the abbreviations of the state you want to look up: MN


In [ ]:
df.head()

,Site ID,Site Type,Site Name,Status,Legacy Satellite ID,Affiliate Name,Health System Name,Regions,Categories,Inactivated Reason,Physical Location: Line 1,Physical Location: Line 2,Physical Location: Line 3,Physical Location: City,Physical Location: State,Physical Location: Zip,Shipping Address: Line 1,Shipping Address: Line 2,Shipping Address: Line 3,Shipping Address: City,Shipping Address: State,Shipping Address: Zip,Latitude,Longitude,County,Site Type.1,On U.S. Military Base,Site Interest: potential volunteers,Site Interest: new or used book donations,Site Interest: hosting a legislator visit,Site Interest: potential site funders,Public Health,Community Health Center,Hospital,Private Practice,IHS,Migrant Population,Military Treatment Facility (MTF),Healthy Steps,Pediatric Residency,...,Report 1: Well-Child Visits 0-5 years over 6 mos,Report 1: Books 0 - 5 years over 6 mos,Report 1: Book Distribution Rate,Total number of new books on hand at sites and satellites,Primary Contact Name,Primary Contact Phone,Primary Contact Email,Medical Consultant Name,Medical Consultant Phone,Medical Consultant Email,Last Submitted Report,Last Submitted Report Date,Current Report,Current Report Status,Current Report Eligibility,ROR+ Museum,ROR+ Library,ROR+ Adult Education/Literacy Council,ROR+ Smart Start/First Steps,ROR+ Healthy Steps,ROR+ College/University,ROR+ Imagination Library,ROR+ Early Head Start,ROR+ Parent-Child Home Partnership,ROR+ Centering Medicine,ROR+ Relevant State Department Agencies,ROR+ Other & Notes,Eligible for funding,W9 received,AAB Account,AAB Balance,AAB Date,Scholastic Account,Scholastic Balance,Scholastic Date,Most Recent Quality Classification Date,Most Recent Quality Classification Assessment,Most Recent Quality Classification Activity,"Site is encouraging parents/caregivers to talk, read, sing, and play with their babies at the WCV’s 0-5 months as instructed by the ROR anticipatory guidance",Site wants to give books at all or some WCVs from 0-5 months
845,482.0,Primary,Indian Health Board of Minneapolis /,Active,NaN,Reach Out and Read Minnesota,NaN,"Jenn's Clinics, West Metro MN, Native American...",ROR American Indian/Alaska Native,NaN,1315 East 24th Street,NaN,NaN,Minneapolis,MN,554043975,"Cara Schnick, Care Coordinator",Indian Health Board of Mpls,1315 E. 24th St,Minneapolis,MN,55404,44.958907,-93.255413,Hennepin,Urban,N,N,Y,Y,Y,NaN,Y,N,N,Y,N,NaN,N,N,...,189.0,133.0,0.70,NaN,Delilah Robb,(612) 721-9800,Delilah.Robb@indianhealthboard.com,Angela Erdrich,(612) 721-9800,aerdrich@ihb-mpls.org,January 2022 Progress Report,2022-02-04,January 2022 Progress Report,Completed,Eligible,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,Y,Y,25954,0.0,2022-02-17,554892,0.0,2022-02-14,2021-09-08,Green,2021-09-08,Y,Y
846,483.0,Primary,Children's Minnesota / Minneapolis,Active,NaN,Reach Out and Read Minnesota,Children's Hospital of MN,"Jenn's Clinics, West Metro MN, Children's Hosp...",NaN,NaN,2530 Chicago Avenue South,#CSC-390,NaN,Minneapolis,MN,554044518,Brian Hall,Children's Hospital,2530 Chicago Ave S CSC390,Minneapolis,MN,55404,44.956687,-93.263005,Hennepin,NaN,N,Y,Y,N,Y,NaN,N,N,N,N,N,NaN,N,N,...,2280.0,1679.0,0.74,NaN,Mary Schnabel,(612) 813-6107,mary.schnabel@childrensmn.org,Mary Schnabel,(612) 813-6107,mary.schnabel@childrensmn.org,July 2021 Progress Report,2021-08-30,January 2022 Progress Report,Not Started,Eligible,N,Y,Y,N,Y,N,N,NaN,NaN,NaN,NaN,NaN,Y,Y,39236,0.0,2022-02-17,465784,0.0,2022-02-14,2019-10-01,Yellow,2019-10-01,Y,Y
847,484.0,Primary,Community-University Health Care Center,Active,NaN,Reach Out and Read Minnesota,NaN,"West Metro MN, Jenn's Clinics, U of M Universi...",NaN,NaN,2001 Bloomington Avenue South,NaN,NaN,Minneapolis,MN,554043074,Michael Aylward,CUHCC,2001 Bloomington Ave S,Minneapolis,MN,55404,44.962069,-93.251831,Hennepin,Urban,N,N,Y,Y,Y,NaN,N,N,N,N,N,NaN,N,N,...,240.0,125.0,0.52,NaN,Michael Aylward,(612) 638-0700,aylwa002@umn.edu,Michael Aylward,(612) 638-0700,aylwa002@umn.edu,July 2021 Progress Report

In [ ]:
FQHC_data_report = files.upload()
FQHC_data_file = next(iter(FQHC_data_report))
FQHC_df = pd.read_excel(FQHC_data_file)

Saving FQHCs_MN.xlsx to FQHCs_MN.xlsx


In [ ]:
# Remove rows which contain grantee information/BHCMISID information
# Bureau of Primary Health Care Health Center Management Information System ID (BHCMISID)

FQHC_df = FQHC_df[pd.isnull(FQHC_df['BHCMISID'])]
# FQHC_df.drop('BHCMISID', axis = 1, inplace = True)
# FQHC_df.columns = FQHC_df.iloc[0]
# FQHC_df = FQHC_df.iloc[1:]
# FQHC_df = FQHC_df[FQHC_df["UDS Number"].str.contains("UDS Number") == False]

In [ ]:
FQHC_df.drop('BHCMISID', axis = 1, inplace = True)

In [ ]:
FQHC_df.columns = FQHC_df.iloc[0]


In [ ]:
FQHC_df = FQHC_df.iloc[1:]

In [ ]:
RHC_data_report = files.upload()
RHC_data_file = next(iter(RHC_data_report))
RHC_df = pd.read_excel(RHC_data_file)

Saving RHCs_MN.xlsx to RHCs_MN.xlsx


In [ ]:
RHC_df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,County State,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53
0,Site Name,Grant #,Site Category,Site Type,Site Sub Type,Address,City,County,Cong. District,State,ZIP Code,Phone,Dash Site Id,Cmn State Nm,"Aitkin County, MN","Becker County, MN","Big Stone County, MN","Brown County, MN","Cass County, MN","Clay County, MN","Clearwater County, MN","Cottonwood County, MN","Crow Wing County, MN","Hubbard County, MN","Jackson County, MN","Kittson County, MN","Koochiching County, MN","Lac qui Parle County, MN","Lake of the Woods County, MN","Lincoln County, MN","Lyon County, MN","Mahnomen County, MN","Marshall County, MN","Meeker County, MN","Mille Lacs County, MN","Morrison County, MN","Murray County, MN","Norman County, MN","Otter Tail County, MN","Pine County, MN","Polk County, MN","Pope County, MN","Red Lake County, MN","Redwood County, MN","Renville County, MN","Sibley County, MN","St. Louis County, MN","Stearns County, MN","Swift County, MN","Todd County, MN","Traverse County, MN","Wabasha County, MN","Wadena County, MN","Yellow Medicine County, MN"
1,APPLETON AREA HEALTH,NaN,Rural Health Clinic,Rural Health Clinic,Rural Health Clinic,30 S Behl St,Appleton,Swift County,Minnesota District 07,MN,56208-1616,320-289-1580,RHC0000076502,Minnesota,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AVERA MEDICAL GROUP LAKE BENTON,NaN,Rural Health Clinic,Rural Health Clinic,Rural Health Clinic,109 E Benton St,Lake Benton,Lincoln County,Minnesota District 07,MN,56149-1309,507-368-4323,RHC0000051787,Minnesota,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AVERA MEDICAL GROUP TYLER,NaN,Rural Health Clinic,Rural Health Clinic,Rural Health Clinic,240 Willow St,Tyler,Lincoln County,Minnesota District 07,MN,56178-1201,507-247-5921,RHC0000021560,Minnesota,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BROOTEN MEDICAL CENTER,NaN,Rural Health Clinic,Rural Health Clinic,Rural Health Clinic,PO BOX 69,Brooten,Pope County,Minnesota District 07,MN,56316-0069,320-346-2272,RHC0000021672,Minnesota,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
RHC_df.columns = RHC_df.iloc[0]

In [ ]:
RHC_df = RHC_df.iloc[1:]

In [ ]:
RHC_df.head()

,Site Name,Grant #,Site Category,Site Type,Site Sub Type,Address,City,County,Cong. District,State,ZIP Code,Phone,Dash Site Id,Cmn State Nm,"Aitkin County, MN","Becker County, MN","Big Stone County, MN","Brown County, MN","Cass County, MN","Clay County, MN","Clearwater County, MN","Cottonwood County, MN","Crow Wing County, MN","Hubbard County, MN","Jackson County, MN","Kittson County, MN","Koochiching County, MN","Lac qui Parle County, MN","Lake of the Woods County, MN","Lincoln County, MN","Lyon County, MN","Mahnomen County, MN","Marshall County, MN","Meeker County, MN","Mille Lacs County, MN","Morrison County, MN","Murray County, MN","Norman County, MN","Otter Tail County, MN","Pine County, MN","Polk County, MN","Pope County, MN","Red Lake County, MN","Redwood County, MN","Renville County, MN","Sibley County, MN","St. Louis County, MN","Stearns County, MN","Swift County, MN","Todd County, MN","Traverse County, MN","Wabasha County, MN","Wadena County, MN","Yellow Medicine County, MN"
1,APPLETON AREA HEALTH,NaN,Rural Health Clinic,Rural Health Clinic,Rural Health Clinic,30 S Behl St,Appleton,Swift County,Minnesota District 07,MN,56208-1616,320-289-1580,RHC0000076502,Minnesota,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AVERA MEDICAL GROUP LAKE BENTON,NaN,Rural Health Clinic,Rural Health Clinic,Rural Health Clinic,109 E Benton St,Lake Benton,Lincoln County,Minnesota District 07,MN,56149-1309,507-368-4323,RHC0000051787,Minnesota,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AVERA MEDICAL GROUP TYLER,NaN,Rural Health Clinic,Rural Health Clinic,Rural Health Clinic,240 Willow St,Tyler,Lincoln County,Minnesota District 07,MN,56178-1201,507-247-5921,RHC0000021560,Minnesota,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BROOTEN MEDICAL CENTER,NaN,Rural Health Clinic,Rural Health Clinic,Rural Health Clinic,PO BOX 69,Brooten,Pope County,Minnesota District 07,MN,56316-0069,320-346-2272,RHC0000021672,Minnesota,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"CARRIS HEALTH - REDWOOD, LLC",NaN,Rural Health Clinic,Rural Health Clinic,Rural Health Clinic,1100 E Broadway St,Redwood Falls,Redwood County,Minnesota District 07,MN,56283-2247,507-637-2985,RHC0000076631,Minnesota,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def county_fix(address):
  address_list = address.split()

In [ ]:
def normalization(address):
  # Make everything lowercase
  address = address.lower()

  # Remove punctuation
  punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
  for element in address:
    if element in punc:
      address = address.replace(element, "")
  # Split address into list
  address_list = address.split()
  count = 0
  for i in address_list:
    if i == ("n" or "north"):
      address_list[count] = "North"
    if i == ("w" or "west"):
      address_list[count] = "West"
    if i == ("s" or "south"):
      address_list[count] = "South"
    if i == ("e" or "east"):
      address_list[count] = "East" 
    if i == ("ne" or "northeast"):
      address_list[count] = "Northeast"
    if i == ("nw" or "northwest"):
      address_list[count] = "Northwest"
    if i == ("se" or "southeast"):
      address_list[count] = "Southeast"
    if i == ("sw" or "southwest"):
      address_list[count] = "Southwest" 
    if i == ("apt" or "unit" or "suite" or "ste"):
      address_list[count] = "#"   
    if i == ("ave" or "av" or "avenue"):
      address_list[count] = "Avenue"
    if i == ("blv" or "blvd" or "boulevard"):
      address_list[count] = "Boulevard"
    if i == ("ct" or "court"):
      address_list[count] = "Court"
    if i == ("dr" or "drv" or "drive"):
      address_list[count] = "Drive"
    if i == ("ln" or "lane"):
      address_list[count] = "Lane"
    if i == ("pl" or "place"):
      address_list[count] = "Place"
    if i == ("rd" or "road"):
      address_list[count] = "Road"
    if i == ("st" or "str" or "street"):
      address_list[count] = "Street"
    if i == "po":
      address_list[count] = "PO"
    if i == ("cty" or "cnty" or "county"):
      address_list[count] = "County"
    if i == ("hwy" or "hway" or "highway"):
      address_list[count] = "Highway"
    if i == "first":
      address_list[count] = "1st"
    if i == "second":
      address_list[count] = "2nd"
    if i == "third":
      address_list[count] = "3rd"
    if i == "fourth":
      address_list[count] = "4th"
    if i == "fifth":
      address_list[count] = "5th"
    if i == "sixth":
      address_list[count] = "6th"
    if i == "seventh":
      address_list[count] = "7th"
    if i == "eigth":
      address_list[count] = "8th"
    if i == "ninth":
      address_list[count] = "9th"
    if i == "tenth":
      address_list[count] = "10th"
    count += 1

  address_rejoin = ' '.join(address_list)
  address_title = address_rejoin.title()
  address_list_characters = list(address_title)
  list_of_str_numbers = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0']
  list_of_cap_letters = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
  # list_of_directional_letters = ['N', 'S']
  # list2_of_directional_letters = ['e', 'w']
  last_letter = len(address_title) - 1
  for i in range(len(address_title)):
    if i != last_letter:
      if address_title[i] in list_of_str_numbers and address_title[i+1] in list_of_cap_letters:
        address_list_characters[i+1] = address_title[i+1].lower()
      # if address_title[i] in list_of_directional_letters and address_title[i+1] in list2_of_directional_letters:
      #   if i+1 == last_letter or address_title[i+2] == " ":
      #     print("I'm working")
      #     address_list_characters[i+1] = address_title[i+1].upper()
      if address_title[i] == "P" and address_title[i+1] == "o":
        if i+1 == last_letter or address_title[i+2] == " ":
          address_list_characters[i+1] = address_title[i+1].upper()
  address_final = ''.join(address_list_characters)
  return address_final

In [ ]:
normalization("1 ne Lake Street")

'1 Northeast Lake Street'

In [ ]:
list_of_normalized_FQHC_addresses = []
list_of_normalized_ROR_addresses =  []
list_of_normalized_RHC_addresses = []

In [ ]:
for i, rows in df.iterrows():
  normalized_ROR_address = normalization(df.loc[i, 'Physical Location: Line 1'])
  list_of_normalized_ROR_addresses.append(normalized_ROR_address)

In [ ]:
for j, rows in FQHC_df.iterrows():
  normalized_FQHC_address = normalization(FQHC_df.loc[j, 'Address'])
  list_of_normalized_FQHC_addresses.append(normalized_FQHC_address)

In [ ]:
for k, rows in RHC_df.iterrows():
  normalized_RHC_address = normalization(RHC_df.loc[k, 'Address'])
  list_of_normalized_RHC_addresses.append(normalized_RHC_address)

In [ ]:
df.insert(11, "ROR Normalized Address", list_of_normalized_ROR_addresses, True)

In [ ]:
FQHC_df.insert(7, "FQHC Normalized Address", list_of_normalized_FQHC_addresses, True)

In [ ]:
RHC_df.insert(6, "RHC Normalized Address", list_of_normalized_RHC_addresses, True)

In [ ]:
RHC_df.head()

,Site Name,Grant #,Site Category,Site Type,Site Sub Type,Address,RHC Normalized Address,City,County,Cong. District,State,ZIP Code,Phone,Dash Site Id,Cmn State Nm,"Aitkin County, MN","Becker County, MN","Big Stone County, MN","Brown County, MN","Cass County, MN","Clay County, MN","Clearwater County, MN","Cottonwood County, MN","Crow Wing County, MN","Hubbard County, MN","Jackson County, MN","Kittson County, MN","Koochiching County, MN","Lac qui Parle County, MN","Lake of the Woods County, MN","Lincoln County, MN","Lyon County, MN","Mahnomen County, MN","Marshall County, MN","Meeker County, MN","Mille Lacs County, MN","Morrison County, MN","Murray County, MN","Norman County, MN","Otter Tail County, MN","Pine County, MN","Polk County, MN","Pope County, MN","Red Lake County, MN","Redwood County, MN","Renville County, MN","Sibley County, MN","St. Louis County, MN","Stearns County, MN","Swift County, MN","Todd County, MN","Traverse County, MN","Wabasha County, MN","Wadena County, MN","Yellow Medicine County, MN"
1,APPLETON AREA HEALTH,NaN,Rural Health Clinic,Rural Health Clinic,Rural Health Clinic,30 S Behl St,30 South Behl Street,Appleton,Swift County,Minnesota District 07,MN,56208-1616,320-289-1580,RHC0000076502,Minnesota,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AVERA MEDICAL GROUP LAKE BENTON,NaN,Rural Health Clinic,Rural Health Clinic,Rural Health Clinic,109 E Benton St,109 East Benton Street,Lake Benton,Lincoln County,Minnesota District 07,MN,56149-1309,507-368-4323,RHC0000051787,Minnesota,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AVERA MEDICAL GROUP TYLER,NaN,Rural Health Clinic,Rural Health Clinic,Rural Health Clinic,240 Willow St,240 Willow Street,Tyler,Lincoln County,Minnesota District 07,MN,56178-1201,507-247-5921,RHC0000021560,Minnesota,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BROOTEN MEDICAL CENTER,NaN,Rural Health Clinic,Rural Health Clinic,Rural Health Clinic,PO BOX 69,PO Box 69,Brooten,Pope County,Minnesota District 07,MN,56316-0069,320-346-2272,RHC0000021672,Minnesota,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"CARRIS HEALTH - REDWOOD, LLC",NaN,Rural Health Clinic,Rural Health Clinic,Rural Health Clinic,1100 E Broadway St,1100 East Broadway Street,Redwood Falls,Redwood County,Minnesota District 07,MN,56283-2247,507-637-2985,RHC0000076631,Minnesota,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.head()

,Site ID,Site Type,Site Name,Status,Legacy Satellite ID,Affiliate Name,Health System Name,Regions,Categories,Inactivated Reason,Physical Location: Line 1,ROR Normalized Address,Physical Location: Line 2,Physical Location: Line 3,Physical Location: City,Physical Location: State,Physical Location: Zip,Shipping Address: Line 1,Shipping Address: Line 2,Shipping Address: Line 3,Shipping Address: City,Shipping Address: State,Shipping Address: Zip,Latitude,Longitude,County,Site Type.1,On U.S. Military Base,Site Interest: potential volunteers,Site Interest: new or used book donations,Site Interest: hosting a legislator visit,Site Interest: potential site funders,Public Health,Community Health Center,Hospital,Private Practice,IHS,Migrant Population,Military Treatment Facility (MTF),Healthy Steps,...,Report 1: Well-Child Visits 0-5 years over 6 mos,Report 1: Books 0 - 5 years over 6 mos,Report 1: Book Distribution Rate,Total number of new books on hand at sites and satellites,Primary Contact Name,Primary Contact Phone,Primary Contact Email,Medical Consultant Name,Medical Consultant Phone,Medical Consultant Email,Last Submitted Report,Last Submitted Report Date,Current Report,Current Report Status,Current Report Eligibility,ROR+ Museum,ROR+ Library,ROR+ Adult Education/Literacy Council,ROR+ Smart Start/First Steps,ROR+ Healthy Steps,ROR+ College/University,ROR+ Imagination Library,ROR+ Early Head Start,ROR+ Parent-Child Home Partnership,ROR+ Centering Medicine,ROR+ Relevant State Department Agencies,ROR+ Other & Notes,Eligible for funding,W9 received,AAB Account,AAB Balance,AAB Date,Scholastic Account,Scholastic Balance,Scholastic Date,Most Recent Quality Classification Date,Most Recent Quality Classification Assessment,Most Recent Quality Classification Activity,"Site is encouraging parents/caregivers to talk, read, sing, and play with their babies at the WCV’s 0-5 months as instructed by the ROR anticipatory guidance",Site wants to give books at all or some WCVs from 0-5 months
845,482.0,Primary,Indian Health Board of Minneapolis /,Active,NaN,Reach Out and Read Minnesota,NaN,"Jenn's Clinics, West Metro MN, Native American...",ROR American Indian/Alaska Native,NaN,1315 East 24th Street,1315 East 24th Street,NaN,NaN,Minneapolis,MN,554043975,"Cara Schnick, Care Coordinator",Indian Health Board of Mpls,1315 E. 24th St,Minneapolis,MN,55404,44.958907,-93.255413,Hennepin,Urban,N,N,Y,Y,Y,NaN,Y,N,N,Y,N,NaN,N,...,189.0,133.0,0.70,NaN,Delilah Robb,(612) 721-9800,Delilah.Robb@indianhealthboard.com,Angela Erdrich,(612) 721-9800,aerdrich@ihb-mpls.org,January 2022 Progress Report,2022-02-04,January 2022 Progress Report,Completed,Eligible,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,Y,Y,25954,0.0,2022-02-17,554892,0.0,2022-02-14,2021-09-08,Green,2021-09-08,Y,Y
846,483.0,Primary,Children's Minnesota / Minneapolis,Active,NaN,Reach Out and Read Minnesota,Children's Hospital of MN,"Jenn's Clinics, West Metro MN, Children's Hosp...",NaN,NaN,2530 Chicago Avenue South,2530 Chicago Avenue South,#CSC-390,NaN,Minneapolis,MN,554044518,Brian Hall,Children's Hospital,2530 Chicago Ave S CSC390,Minneapolis,MN,55404,44.956687,-93.263005,Hennepin,NaN,N,Y,Y,N,Y,NaN,N,N,N,N,N,NaN,N,...,2280.0,1679.0,0.74,NaN,Mary Schnabel,(612) 813-6107,mary.schnabel@childrensmn.org,Mary Schnabel,(612) 813-6107,mary.schnabel@childrensmn.org,July 2021 Progress Report,2021-08-30,January 2022 Progress Report,Not Started,Eligible,N,Y,Y,N,Y,N,N,NaN,NaN,NaN,NaN,NaN,Y,Y,39236,0.0,2022-02-17,465784,0.0,2022-02-14,2019-10-01,Yellow,2019-10-01,Y,Y
847,484.0,Primary,Community-University Health Care Center,Active,NaN,Reach Out and Read Minnesota,NaN,"West Metro MN, Jenn's Clinics, U of M Universi...",NaN,NaN,2001 Bloomington Avenue South,2001 Bloomington Avenue South,NaN,NaN,Minneapolis,MN,554043074,Michael Aylward,CUHCC,2001 Bloomington Ave S,Minneapolis,MN,55404,44.962069,-93.251831,Hennepin,Urban,N,N,Y,Y,Y,NaN,N,N,N,N,N,NaN,N,...,240.0,125.0,0.52,NaN,Michael Aylward,(612) 638-0700,aylwa002@umn.ed

In [ ]:
writer = pd.ExcelWriter('Potential_ROR_FQHC_RHC_Matches.xlsx', engine='xlsxwriter')
#Initiate new lists for new spreadsheet

# ROR LISTS compared to FQHC
ROR_address_data_compared_to_FQHC = []
ROR_normalized_address_data_compared_to_FQHC = []
ROR_ID_data_compared_to_FQHC = []
ROR_name_data_compared_to_FQHC = []
ROR_county_data_compared_to_FQHC = []
ROR_city_data_compared_to_FQHC = []

# FQHC LISTS
FQHC_address_data = []
FQHC_normalized_address_data = []
FQHC_name_data = []
FQHC_county_data = []
FQHC_city_data = []
FQHC_site_number_data = []

# ROR LISTS compared to RHC
ROR_address_data_compared_to_RHC = []
ROR_normalized_address_data_compared_to_RHC = []
ROR_ID_data_compared_to_RHC = []
ROR_name_data_compared_to_RHC = []
ROR_county_data_compared_to_RHC = []
ROR_city_data_compared_to_RHC = []

# RHC LISTS
RHC_address_data = []
RHC_normalized_address_data = []
RHC_name_data = []
RHC_county_data = []
RHC_city_data = []
RHC_site_number_data = []

fuzzy_score_FQHC = []
fuzzy_score_RHC = []

In [ ]:
for i, rows in df.iterrows():
  ROR_address = df.loc[i, 'Physical Location: Line 1']
  ROR_normalized_address = df.loc[i, 'ROR Normalized Address']
  ROR_name = df.loc[i, 'Site Name']
  ROR_site_id = df.loc[i, 'Site ID']
  ROR_county = df.loc[i, 'County']
  ROR_city = df.loc[i, 'Physical Location: City']

  ##########################
  ####### CHECK FQHC #######
  ##########################
  for j, rows, in FQHC_df.iterrows():
    FQHC_address = FQHC_df.loc[j, 'Address']
    FQHC_normalized_address = FQHC_df.loc[j, 'FQHC Normalized Address']
    FQHC_name = FQHC_df.loc[j, 'Satellite Name']
    FQHC_county = FQHC_df.loc[j, 'County']
    FQHC_city = FQHC_df.loc[j, 'City']
    FQHC_site_number = FQHC_df.loc[j, 'BPHC Site Number']

    # Generate ratios for Address comparions
    # FUZZY TOKEN looks for tokens (words) in any order
    fuzzy_ratio_token_address = fuzz.token_sort_ratio(ROR_normalized_address, FQHC_normalized_address)

    # FUZZY RATIO PARTIAL looks for a subset of words within a larger set -- need to check which is the master set
    fuzzy_ratio_partial_address = fuzz.partial_ratio(ROR_normalized_address, FQHC_normalized_address)
    # fuzzy_ratio_partial_address = fuzz.partial_ratio(FQHC_normalized_address, ROR_normalized_address)

    # Generate ratios for county name comparisons
    fuzzy_ratio_partial_county = fuzz.partial_ratio(ROR_county, FQHC_county)


    # COMPARE TOKEN SORT FOR ADDRESSES
    if fuzzy_ratio_token_address >= 95 and fuzzy_ratio_partial_county >= 80 and (ROR_city == FQHC_city):
      ROR_ID_data_compared_to_FQHC.append(ROR_site_id)
      FQHC_site_number_data.append(FQHC_site_number)
      ROR_address_data_compared_to_FQHC.append(ROR_address)
      ROR_normalized_address_data_compared_to_FQHC.append(ROR_normalized_address)
      FQHC_normalized_address_data.append(FQHC_normalized_address)
      FQHC_address_data.append(FQHC_address)
      ROR_county_data_compared_to_FQHC.append(ROR_county)
      FQHC_county_data.append(FQHC_county)
      ROR_name_data_compared_to_FQHC.append(ROR_name)
      FQHC_name_data.append(FQHC_name)
      ROR_city_data_compared_to_FQHC.append(ROR_city)
      FQHC_city_data.append(FQHC_city)
      fuzzy_score_FQHC.append(fuzzy_ratio_token_address)
      print(ROR_address, FQHC_address, fuzzy_ratio_token_address, ROR_city, FQHC_city, ROR_county, FQHC_county)
    
    # COMPARE PARTIAL RATIO FOR ADDRESSES
    elif fuzzy_ratio_partial_address >= 95 and fuzzy_ratio_partial_county >= 80 and (ROR_city == FQHC_city):
      ROR_ID_data_compared_to_FQHC.append(ROR_site_id)
      FQHC_site_number_data.append(FQHC_site_number)
      ROR_address_data_compared_to_FQHC.append(ROR_address)
      ROR_normalized_address_data_compared_to_FQHC.append(ROR_normalized_address)
      FQHC_normalized_address_data.append(FQHC_normalized_address)
      FQHC_address_data.append(FQHC_address)
      ROR_county_data_compared_to_FQHC.append(ROR_county)
      FQHC_county_data.append(FQHC_county)
      ROR_name_data_compared_to_FQHC.append(ROR_name)
      FQHC_name_data.append(FQHC_name)
      ROR_city_data_compared_to_FQHC.append(ROR_city)
      FQHC_city_data.append(FQHC_city)
      fuzzy_score_FQHC.append(fuzzy_ratio_partial_address)
      print(ROR_address, FQHC_address, fuzzy_ratio_partial_address, ROR_city, FQHC_city, ROR_county, FQHC_county)

  ##########################
  ####### CHECK RHC ########
  ##########################

  for k, rows, in RHC_df.iterrows():
    RHC_address = RHC_df.loc[k, 'Address']
    RHC_normalized_address = RHC_df.loc[k, 'RHC Normalized Address']
    RHC_name = RHC_df.loc[k, 'Site Name']
    RHC_county = RHC_df.loc[k, 'County']
    RHC_city = RHC_df.loc[k, 'City']
    RHC_site_number = RHC_df.loc[k, 'Dash Site Id']

    # Generate ratios for Address comparions
    # FUZZY TOKEN looks for tokens (words) in any order
    fuzzy_ratio_token_address = fuzz.token_sort_ratio(ROR_normalized_address, RHC_normalized_address)

    # FUZZY RATIO PARTIAL looks for a subset of words within a larger set -- need to check which is the master set
    fuzzy_ratio_partial_address = fuzz.partial_ratio(ROR_normalized_address, RHC_normalized_address)
    # fuzzy_ratio_partial_address = fuzz.partial_ratio(FQHC_normalized_address, ROR_normalized_address)

    # Generate ratios for county name comparisons
    fuzzy_ratio_partial_county = fuzz.partial_ratio(ROR_county, RHC_county)


    # COMPARE TOKEN SORT FOR ADDRESSES
    if fuzzy_ratio_token_address >= 95 and fuzzy_ratio_partial_county >= 80 and (ROR_city == RHC_city):
      ROR_ID_data_compared_to_RHC.append(ROR_site_id)
      RHC_site_number_data.append(RHC_site_number)
      ROR_address_data_compared_to_RHC.append(ROR_address)
      ROR_normalized_address_data_compared_to_RHC.append(ROR_normalized_address)
      RHC_normalized_address_data.append(RHC_normalized_address)
      RHC_address_data.append(RHC_address)
      ROR_county_data_compared_to_RHC.append(ROR_county)
      RHC_county_data.append(RHC_county)
      ROR_name_data_compared_to_RHC.append(ROR_name)
      RHC_name_data.append(RHC_name)
      ROR_city_data_compared_to_RHC.append(ROR_city)
      RHC_city_data.append(RHC_city)
      fuzzy_score_RHC.append(fuzzy_ratio_token_address)
    
    # COMPARE PARTIAL RATIO FOR ADDRESSES
    elif fuzzy_ratio_partial_address >= 95 and fuzzy_ratio_partial_county >= 80 and (ROR_city == RHC_city):
      ROR_ID_data_compared_to_RHC.append(ROR_site_id)
      RHC_site_number_data.append(RHC_site_number)
      ROR_address_data_compared_to_RHC.append(ROR_address)
      ROR_normalized_address_data_compared_to_RHC.append(ROR_normalized_address)
      RHC_normalized_address_data.append(RHC_normalized_address)
      RHC_address_data.append(RHC_address)
      ROR_county_data_compared_to_RHC.append(ROR_county)
      RHC_county_data.append(RHC_county)
      ROR_name_data_compared_to_RHC.append(ROR_name)
      RHC_name_data.append(RHC_name)
      ROR_city_data_compared_to_RHC.append(ROR_city)
      RHC_city_data.append(RHC_city)
      fuzzy_score_RHC.append(fuzzy_ratio_partial_address)

1315 East 24th Street 1315 E 24th St 100 Minneapolis Minneapolis Hennepin Hennepin County
2001 Bloomington Avenue South 2001 Bloomington Ave 100 Minneapolis Minneapolis Hennepin Hennepin County
614 South 3rd Street 614 S 3rd St 100 Minneapolis Minneapolis Hennepin Hennepin County
1313 Penn Avenue  N. 1313 Penn Ave N 100 Minneapolis Minneapolis Hennepin Hennepin County
513 5th Avenue West 516 5th Ave W 95 Grand Marais Grand Marais Cook Cook County
513 5th Avenue West 513 5th Ave W 100 Grand Marais Grand Marais Cook Cook County
PO Box 428, 62 Upper Road 62 Upper Rd 100 Grand Portage Grand Portage Cook Cook County
1213 E Franklin Ave 1213 E Franklin Ave 100 Minneapolis Minneapolis Hennepin Hennepin County
1213 E Franklin Ave 1113 E Franklin Ave Ste 103 96 Minneapolis Minneapolis Hennepin Hennepin County
1026 West 7th Street 1026 7th St W 100 Saint Paul Saint Paul Ramsey Ramsey County
10081 Dogwood St. NW 10081 Dogwood St NW STE 100 100 Coon Rapids Coon Rapids Anoka Anoka County
425 20th A

In [ ]:
#ROR_FQHC_df = pd.DataFrame(list(zip(ROR_ID_data, ROR_address_data, ROR_shipping_address_data_3, FQHC_address_data, ROR_city_data, FQHC_city_data, ROR_county_data, FQHC_county_data, ROR_name_data, FQHC_name_data, ROR_shipping_address_data_1, ROR_shipping_address_data_2)), columns = ['Site ID', 'ROR Address', 'ROR Shipping Address: Line 3', 'FQHC Address - Possible Match', 'ROR City', 'FQHC City', 'ROR County', 'FQHC County', 'ROR Site Name', 'Satellite Name', 'ROR Shipping Address: Line 1', 'ROR Shipping Address: Line 2'])

In [ ]:
ROR_FQHC_df = pd.DataFrame(list(zip(ROR_ID_data_compared_to_FQHC, ROR_normalized_address_data_compared_to_FQHC, FQHC_normalized_address_data, fuzzy_score_FQHC, ROR_address_data_compared_to_FQHC, FQHC_address_data, ROR_city_data_compared_to_FQHC, FQHC_city_data, ROR_county_data_compared_to_FQHC, FQHC_county_data, ROR_name_data_compared_to_FQHC, FQHC_name_data, FQHC_site_number_data)), columns = ['Site ID', 'Normalized ROR Address', 'Normalized FQHC Address', 'Fuzzy Score', 'ROR Address', 'FQHC Address - Possible Match', 'ROR City', 'FQHC City', 'ROR County', 'FQHC County', 'ROR Site Name', 'FQHC Satellite Name', 'FQHC Site Number'])

In [ ]:
ROR_RHC_df = pd.DataFrame(list(zip(ROR_ID_data_compared_to_RHC, ROR_normalized_address_data_compared_to_RHC, RHC_normalized_address_data, fuzzy_score_RHC, ROR_address_data_compared_to_RHC, RHC_address_data, ROR_city_data_compared_to_RHC, RHC_city_data, ROR_county_data_compared_to_RHC, RHC_county_data, ROR_name_data_compared_to_RHC, RHC_name_data, RHC_site_number_data)), columns = ['Site ID', 'Normalized ROR Address', 'Normalized RHC Address', 'Fuzzy Score', 'ROR Address', 'RHC Address - Possible Match', 'ROR City', 'RHC City', 'ROR County', 'RHC County', 'ROR Site Name', 'RHC Satellite Name', 'RHC Site Number'])

In [ ]:
ROR_FQHC_df.head()

,Site ID,Normalized ROR Address,Normalized FQHC Address,Fuzzy Score,ROR Address,FQHC Address - Possible Match,ROR City,FQHC City,ROR County,FQHC County,ROR Site Name,FQHC Satellite Name,FQHC Site Number
0,482.0,1315 East 24th Street,1315 East 24th Street,100,1315 East 24th Street,1315 E 24th St,Minneapolis,Minneapolis,Hennepin,Hennepin County,Indian Health Board of Minneapolis /,"INDIAN HEALTH BOARD OF MINNEAPOLIS, INC.",BPS-H80-004388
1,484.0,2001 Bloomington Avenue South,2001 Bloomington Avenue,100,2001 Bloomington Avenue South,2001 Bloomington Ave,Minneapolis,Minneapolis,Hennepin,Hennepin County,Community-University Health Care Center,COMMUNITY-UNIVERSITY HEALTH CARE CENTER,BPS-H80-005145
2,2070.0,614 South 3rd Street,614 South 3rd Street,100,614 South 3rd Street,614 S 3rd St,Minneapolis,Minneapolis,Hennepin,Hennepin County,People Serving People,People Serving People,BPS-H80-001449
3,2372.0,1313 Penn Avenue North,1313 Penn Avenue North,100,1313 Penn Avenue N.,1313 Penn Ave N,Minneapolis,Minneapolis,Hennepin,Hennepin County,North Point Health and Wellness Center /,DBA NORTHPOINT HEALTH AND WELLNESS,BPS-H80-003695
4,2474.0,513 5th Avenue West,516 5th Avenue West,95,513 5th Avenue West,516 5th Ave W,Grand Marais,Grand Marais,Cook,Cook County,Sawtooth Mountain Clinic /,Temporary Site - Grand Marais Fire Hall,BPS-H80-031351


In [ ]:
ROR_FQHC_df = ROR_FQHC_df.drop_duplicates()

In [ ]:
ROR_FQHC_df = ROR_FQHC_df.set_index('Site ID')

In [ ]:
ROR_FQHC_df = ROR_FQHC_df.sort_values(by='Fuzzy Score', ascending=False)

In [ ]:
ROR_FQHC_df.head()

,Normalized ROR Address,Normalized FQHC Address,Fuzzy Score,ROR Address,FQHC Address - Possible Match,ROR City,FQHC City,ROR County,FQHC County,ROR Site Name,FQHC Satellite Name,FQHC Site Number
Site ID,,,,,,,,,,,,
482.0,1315 East 24th Street,1315 East 24th Street,100,1315 East 24th Street,1315 E 24th St,Minneapolis,Minneapolis,Hennepin,Hennepin County,Indian Health Board of Minneapolis /,"INDIAN HEALTH BOARD OF MINNEAPOLIS, INC.",BPS-H80-004388
484.0,2001 Bloomington Avenue South,2001 Bloomington Avenue,100,2001 Bloomington Avenue South,2001 Bloomington Ave,Minneapolis,Minneapolis,Hennepin,Hennepin County,Community-University Health Care Center,COMMUNITY-UNIVERSITY HEALTH CARE CENTER,BPS-H80-005145
12253.0,916 Rice Street,916 Rice Street,100,916 Rice St,916 Rice St,Saint Paul,Saint Paul,Ramsey,Ramsey County,Open Cities Health Center-Rice St. /,Open Cities North End Clinic,BPS-H80-018066
9841.0,895 East 7th Street,895 7th Street East,100,895 East Seventh Street,895 7th St E,Saint Paul,Saint Paul,Ramsey,Ramsey County,Minnesota Community Care / East Side Family C...,MN Community Care - East Side Clinic,BPS-H80-011405
9826.0,153 Cesar Chavez Street,153 Cesar Chavez Street,100,153 Cesar Chavez Street,153 Cesar Chavez St,Saint Paul,Saint Paul,Ramsey,Ramsey County,Minnesota Community Care / La Clinica,MN Community Care - La Clinica,BPS-H80-000234


In [ ]:
ROR_RHC_df.head()

,Site ID,Normalized ROR Address,Normalized RHC Address,Fuzzy Score,ROR Address,RHC Address - Possible Match,ROR City,RHC City,ROR County,RHC County,ROR Site Name,RHC Satellite Name,RHC Site Number
0,2553.0,49725 County 83,49725 County 83,100,49725 Cty 83,49725 County 83,Staples,Staples,Todd,Todd County,Lakewood Health System/Staples /,LAKEWOOD CLINIC STAPLES,RHC0000051772
1,2554.0,1233 South Highway 10,1233 Highway 10 South,100,1233 South Highway 10,1233 Highway 10 S,Motley,Motley,Morrison,Morrison County,Lakewood Health System/Motley,LAKEWOOD CLINIC MOTLEY,RHC0000051770
2,2555.0,653 Pillsbury Street North,653 Pillsbury Street North,100,653 Pillsbury Street N,653 Pillsbury St N,Pillager,Pillager,Cass,Cass County,Lakewood Health System/Pillager /,LAKEWOOD CLINIC PILLAGER,RHC0000051771
3,2557.0,30535 241st Avenue,30535 241st Avenue,100,30535 241st Avenue,30535 241st Ave,Browerville,Browerville,Todd,Todd County,Lakewood Health System/Browerville,LAKEWOOD CLINIC BROWERVILLE,RHC0000051769
4,2684.0,1100 East Broadway,1100 East Broadway Street,100,1100 East Broadway,1100 E Broadway St,Redwood Falls,Redwood Falls,Redwood,Redwood County,Carris Health - Redwood Falls /,"CARRIS HEALTH - REDWOOD, LLC",RHC0000076631


In [ ]:
ROR_RHC_df = ROR_RHC_df.drop_duplicates()

In [ ]:
ROR_RHC_df = ROR_RHC_df.set_index('Site ID')

In [ ]:
ROR_RHC_df = ROR_RHC_df.sort_values(by='Fuzzy Score', ascending=False)

In [ ]:
ROR_RHC_df.head()

,Normalized ROR Address,Normalized RHC Address,Fuzzy Score,ROR Address,RHC Address - Possible Match,ROR City,RHC City,ROR County,RHC County,ROR Site Name,RHC Satellite Name,RHC Site Number
Site ID,,,,,,,,,,,,
2553.0,49725 County 83,49725 County 83,100,49725 Cty 83,49725 County 83,Staples,Staples,Todd,Todd County,Lakewood Health System/Staples /,LAKEWOOD CLINIC STAPLES,RHC0000051772
2554.0,1233 South Highway 10,1233 Highway 10 South,100,1233 South Highway 10,1233 Highway 10 S,Motley,Motley,Morrison,Morrison County,Lakewood Health System/Motley,LAKEWOOD CLINIC MOTLEY,RHC0000051770
18103.0,308 8th Street North,308 8th Street North,100,308 8th Street North,308 8th St N,Mountain Lake,Mountain Lake,Cottonwood,Cottonwood County,Sanford Health / Mountain Lake,SANFORD CLINIC MOUNTAIN LAKE,RHC0000077396
17891.0,200 West 1st Street,200 West 1st Street,100,200 West First Street,200 W 1st St,Paynesville,Paynesville,Stearns,Stearns County,CentraCare Paynesville Family Medicine /,CENTRACARE HEALTH - PAYNESVILLE CLINIC,RHC0000052014
12304.0,505 Main Street,505 Main Street,100,505 Main Street,505 Main St,Henderson,Henderson,Sibley,Sibley County,Ridgeview Henderson Clinic /,RIDGEVIEW HENDERSON CLINIC,RHC0000022626


In [ ]:
ROR_FQHC_df.to_excel(writer, sheet_name = "Potential ROR-FQHC Matches")

In [ ]:
ROR_RHC_df.to_excel(writer, sheet_name = "Potential ROR-RHC Matches")

In [ ]:
writer.save()
files.download('Potential_ROR_FQHC_RHC_Matches.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>